#Software Exercise 3

Instructions:
1. Sign the Honor Pledge.
2. In addition to your own notes and books, online resources can also be used but you have to provide reference(s).

##Honor Pledge

I pledge on my honor that I have not given or received any unauthorized assistance on this exam/assignment.

Enter your name/Date: John Danielle T Castor / April 22, 2023


---


An online computer system for trading stocks needs to process orders of the form "buy 100 shares at $\$x$ each" or "sell 100 shares at $\$y$ each." A buy order for $\$x$ can only be processed if there is an existing sell order with price $\$y$ such that $y ≤ x$. Likewise, a sell order for $\$y$ can only be processed if there is an existing buy order with price $\$x$ such that $y \le x$. If a buy or sell order is entered but cannot be processed, it must wait for a future order that allows it to be processed. 

a. Describe a scheme that allows buy and sell orders to be entered in $O(\log n)$ time, independent of whether or not they can be immediately processed.

b. Write a Python program that can process a sequence of stock buy and sell orders as described.

Hint: Use priorities queues.

Link to the Priority Queue implementation: 
https://drive.google.com/file/d/1DUrcikkjWmc9EjMTyj_KGOto7ij9mSbI/view?usp=sharing





In [ ]:
"""
a. -An implementation for the desired system is to sort the sell and buy orders in increasing and decreasing prices, respectively.
-Arrange the two types of orders in two separate data structures, i.e. Priority Queue (PQ) via Binary Heap so that the orders can be entered (insert method) in O(log n) worst-case time.
-The prices of the top elements can then be compared:
    1. If sell price is > buy price, then just add to PQ.
    2. Else, proceed with the transaction, gain computation, and update PQs (and elements) accordingly.
-The above implementation prioritizes the buy orders (highest) since sell price < buy price, if same prices- like FIFO; also PQ via Binary Heap is given
-Gain computation- <less # share(s)> * (<sell price> - <buy price>)


-Sample case analyses
1. If current order is Buy
Sell price <=, sell shares >
Sell: 20price 100shares
Buy: 50price 50shares
Gain: 50(20-50)=-1500

Sell price <=, sell shares <
Sell: 20price 20shares
Buy: 50price 50shares
Gain: 20(20-50)=-600

Sell price <=, sell shares =
Sell: 20price 20shares
Buy: 50price 20shares
Gain: 20(20-50)=-600

2. If current order is Sell
Sell price <=, buy shares >
Buy: 50price 100shares
Sell: 20price 50shares
Gain: 50(20-50)=-1500

Sell price <=, buy shares <
Buy: 50price 20shares
Sell: 20price 50shares
Gain: 20(20-50)=-600

Sell price <=, sell shares =
Buy: 50price 20shares
Sell: 20price 20shares
Gain: 20(20-50)=-600

-Sample input series
PQS1- sell 100shares 100price
PQB1- buy 50shares 50price
PQB2(ubos)- buy 50shares 200price
PQS1(update)- sell 50shares 100price
PQS2- sell 20shares 20price
PQB1(update)- buy 30shares 50price
"""

In [ ]:
# YOUR CODE HERE
#b. Buy orders are prioritized (highest), same prices- like FIFO; PQ via Binary Heap is given
import re

class PriorityQueueBase:                            #Abstract base class for a priority queue

  #Nested _Item class
  class _Item:                                      #Lightweight composite to store priority queue items
    __slots__ = '_key', '_value'

    def __init__(self, k, v):
      self._key = k
      self._value = v

    def __lt__(self, other):
      return self._key < other._key                 #Compare items based on their keys

    def __repr__(self):                             #Changed from tuple to list
      return '[{0},{1}]'.format(self._key, self._value)


  #Public behaviors
  def is_empty(self):                               #Concrete (inverse of abstract?) method assuming abstract len
    #Return True if the priority queue is empty
    return len(self) == 0

  def __len__(self):
    #Return the number of items in the priority queue
    raise NotImplementedError('must be implemented by subclass')


class HeapPriorityQueue(PriorityQueueBase):         #Base class defines _Item; FIFO if same key
                                                    #A min-oriented priority queue implemented with a binary heap

  #Nonpublic behaviors
  def _parent(self, j):                             #j- index?
    return (j-1) // 2

  def _left(self, j):
    return 2*j + 1

  def _right(self, j):
    return 2*j + 2

  def _has_left(self, j):
    return self._left(j) < len(self._data)          #Index beyond end of list?

  def _has_right(self, j):
    return self._right(j) < len(self._data)         #Index beyond end of list?

  def _swap(self, i, j):
    #Swap the elements at indices i and j of array
    self._data[i], self._data[j] = self._data[j], self._data[i]

  def _upheap(self, j):
    parent = self._parent(j)
    if j > 0 and self._data[j] < self._data[parent]:
      self._swap(j, parent)
      self._upheap(parent)                          #Recur at position of parent
  
  def _downheap(self, j):
    if self._has_left(j):
      left = self._left(j)
      small_child = left                            #Although right may be smaller
      if self._has_right(j):
        right = self._right(j)
        if self._data[right] < self._data[left]:
          small_child = right
      if self._data[small_child] < self._data[j]:
        self._swap(j, small_child)
        self._downheap(small_child)                 #Recur at position of small_child

  #Public behaviors
  def __init__(self):
    #Create a new empty Priority Queue
    self._data = []

  def __len__(self):
    #Return the number of items in the priority queue
    return len(self._data)

  def add(self, key, value):
    #Add a key-value pair to the priority queue
    self._data.append(self._Item(key, value))
    self._upheap(len(self._data) - 1)               #Upheap newly added position
  
  def min(self):
    #Return but do not remove (k,v) tuple (list) with minimum key
    #Raise Empty exception if empty

    #if self.is_empty():
      #raise Empty('Priority queue is empty.')
    item = self._data[0]
    return [item._key, item._value]

  def remove_min(self):
    #Remove and return (k,v) tuple (list) with minimum key
    #Raise Empty exception if empty.

    #if self.is_empty():
      #raise Empty('Priority queue is empty.')
    self._swap(0, len(self._data) - 1)           # put minimum item at the end
    item = self._data.pop()                      # and remove it from the list;
    self._downheap(0)                            # then fix new root
    return [item._key, item._value]


if __name__ == '__main__':
  PQbuy = HeapPriorityQueue()
  PQsell = HeapPriorityQueue()
  answer = 0
  input1 = []
  current = [[]]

  #Input format- ["buy/sell intx share(s) at inty each", ...], assume input format is correct
    #input3 = ["buy 100 share(s) at 20 each", "buy 20 share(s) at 24 each", "buy 200 share(s) at 36 each", "sell 150 share(s) at 30 each"]
    #input4 = ["buy 100 share(s) at 10 each", "sell 1000 share(s) at 20 each"]
  
  def AddToPQ(input2):
    global input1
    if input2[0:4] == "buy ":
      nums = [int(nums) for nums in re.findall(r'-?\d+\.?\d*', input2)]
      current[0] = [nums[1],nums[0],0]          #0- buy, 1- sell; Price-share
      input1.pop()
    elif input2[0:4] == "sell":
      nums = [int(nums) for nums in re.findall(r'-?\d+\.?\d*', input2)]
      current[0] = [nums[1],nums[0],1]
      input1.pop()
    StockProcess()
    
  def StockProcess():                                                       #Assume cash per transaction is limited to 9999999999
    global answer
    if PQbuy.is_empty() and PQsell.is_empty():
      if current[0][2] == 0:
        PQbuy.add(9999999999 - current[0][0], current[0][1])                #Price-share; 10 9s
      elif current[0][2] == 1:
        PQsell.add(current[0][0], current[0][1])
      answer += 0
    elif ((PQbuy.is_empty() and current[0][2] == 1) or (PQsell.is_empty() and current[0][2] == 0)):
      if current[0][2] == 0:
        PQbuy.add(9999999999 - current[0][0], current[0][1])                #Price-share
      elif current[0][2] == 1:
        PQsell.add(current[0][0], current[0][1])
      answer += 0
    else:
      if current[0][2] == 0:                                                #Buy
        if PQsell.min()[0] <= current[0][0]:                                #Sell price <=
          if PQsell.min()[1] > current[0][1]:                               #Sell share >
            answer += current[0][1] * (PQsell.min()[0] - current[0][0])
            PQsell.add(PQsell.min()[0], (PQsell.min()[1] - current[0][1]))
            PQsell.remove_min()
          elif PQsell.min()[1] < current[0][1]:                             #Sell share <
            answer += PQsell.min()[1] * (PQsell.min()[0] - current[0][0])
            PQbuy.add(9999999999 - current[0][0], (current[0][1] - PQsell.min()[1]))      #Price-share
            PQsell.remove_min()
          elif PQsell.min()[1] == current[0][1]:                            #Sell share =
            answer += current[0][1] * (PQsell.min()[0] - current[0][0])
            PQsell.remove_min()
        else:                                                               #Sell price >
            PQbuy.add(9999999999 - current[0][0], current[0][1])            #Price-share

      elif current[0][2] == 1:                                              #Sell
        if (-(PQbuy.min()[0] - 9999999999)) >= current[0][0]:               #Buy price >= (sell price <=)
          if PQbuy.min()[1] > current[0][1]:                                #Buy share >
            answer -= current[0][1] * (-(PQbuy.min()[0] - 9999999999) - current[0][0])
            PQbuy.add(9999999999 - PQbuy.min()[0], (PQbuy.min()[1] - current[0][1]))
            PQbuy.remove_min()
          elif PQbuy.min()[1] < current[0][1]:                              #Buy share <
            answer -= PQbuy.min()[1] * (-(PQbuy.min()[0] - 9999999999) - current[0][0])
            PQsell.add(current[0][0], (current[0][1] - PQbuy.min()[1]))     #Price-share
            PQbuy.remove_min()
          elif PQbuy.min()[1] == current[0][1]:          
            answer -= current[0][1] * (-(PQbuy.min()[0] - 9999999999) - current[0][0])
            PQbuy.remove_min()
        else:                                                               #Sell price >
            PQsell.add(current[0][0], current[0][1])                        #Price-share
            
    #Cleanup
            
    # return 0
      

  #Infinite loop, only 1 input at a time, assume input format and timing is correct
  while True:
    if input1 == []:                            #F5 to run in VSCode
      inp = str(input())
      if inp == "":
        break
      else:
        input1.append(inp)
        AddToPQ(input1[0])
        print("Gain: " + str(answer))
        print("Current: " + str(current))
        print("PQsell: " + str(PQsell._data))
        print("PQbuy: " + str(PQbuy._data))     #Price = 9999999999-PQbuy._data[x][0]